In [ ]:
import torchvision
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.set_warn_always(False)
%load_ext autoreload
%autoreload 2

# load data

In [ ]:
from data_loader.loader import get_cat_dog, get_fashion_mnist, sample_from
seed = 0
print("getting fashion mnist data")
# training data sources; fashion_mnist has 10 class, we filter it with e.g., [5,0] to make 5th class as 1, 0th class as 0, turning it into a binary problem
mnist_train_loader_1, sandals_val_loader = get_fashion_mnist([5,0], batch_size=16, seed = 0, portion_training=0.95) # sandals
mnist_train_loader_2, sneakers_val_loader = get_fashion_mnist([7,1], batch_size=16, seed = 0, portion_training=0.95) # sneakers
mnist_train_loader_4, bag_val_loader = get_fashion_mnist([8,4], batch_size=16, seed = 0, portion_training=0.95) # bag

# evaluation
mnist_train_loader_5, shirt_val_loader = get_fashion_mnist([6,4], batch_size=16, portion_training=0.95, seed = 0) # shirt
mnist_train_loader_3, boots_val_loader = get_fashion_mnist([9,2], batch_size=16, seed = 0, portion_training=0.95) # boots

print("getting cat dog data")
cat_dog_train_loader, cat_dog_val_loader = get_cat_dog(batch_size=16, seed = 0, portion_training=0.95) # not really useful cat and dog data (for classifying fashion mnist)

sandals_loader = mnist_train_loader_1
sneakers_loader = mnist_train_loader_2
bag_loader = mnist_train_loader_4

boots_loader=mnist_train_loader_3
shirt_loader=mnist_train_loader_5


# check data

In [ ]:
# check if loaded properly
for data in cat_dog_train_loader.dataset:
    plt.imshow(data[0][0])
    plt.show()
    break

# BO function

In [ ]:
from BO import iterative_loop, get_BO_plots
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound

def run_BO(all_loaders, validaton_dataloader, iterations, num_epochs=20, printout=False):
    print("running BO...")
    X, observations, gp = iterative_loop(all_loaders, validaton_dataloader, num_epochs=num_epochs, iterations=iterations, printout=printout)
    BO_to_plot = get_BO_plots(observations) # BO results
    naive_combine = BO_to_plot[0] # naive mixing result is the first iteration result of BO

    # plot model performance as BO progresses...
    plt.plot(range(len(BO_to_plot)), BO_to_plot, c="blue", alpha=0.3, label="BO on mixing ratio")
    plt.axhline(naive_combine, linestyle="--", c="red", label="sample from each data source equally")
    plt.xlabel("BO iterations")
    plt.ylabel("accuracy on evaluation task")
    plt.legend()
    plt.show()

    # plot posterior
    # posterior_acc = []
    # for x in np.linspace(0,1,100):
    #     posterior_acc.append(gp.posterior(torch.Tensor([[x,1-x]])).mean.item())
        
    # plt.plot(np.linspace(0,1,100), posterior_acc)
    # plt.xlabel("mixing ratio (percentage on cats and dogs)")
    # plt.ylabel("accuracy")
    # plt.title("evaluation ratio : 1.0 cats and dogs")
    # plt.show()

    def get_optimal_mixture_from_GP_posterior():
        UCB = UpperConfidenceBound(gp, beta=0.0)
        bounds = torch.stack([torch.zeros(len(all_loaders)), torch.ones(len(all_loaders))]) # need to change the bounds for parameters
        A = [1.0] * len(all_loaders)
        x = list(range(len(all_loaders)))
        candidate, acq_value = optimize_acqf(
            UCB, bounds=bounds, q=1, num_restarts=20, raw_samples=30,
            equality_constraints = [(torch.tensor(x), torch.tensor(A), 1)]
        )
        return candidate
    

    def get_best_observation_mixture():
        
        # Find the index in list B that has the highest value
        highest_index = observations.index(max(observations))
        
        # Return the corresponding item in list A
        return X[highest_index]

    
    print("best mixture found in BO iterations is: ", get_best_observation_mixture())
    
    return X, observations, gp

# run BO

In [ ]:
from image_training import train
from helper import get_data_from_mixing_ratio

evaluation_task_data_ratio = 0.3
print("train data: shirt, boots ")
print("test data ratio: shirt {}%, boots {}%".format(evaluation_task_data_ratio, 1-evaluation_task_data_ratio))

# data for training, we can change this composition if necessary with other loaders
train_loaders = [shirt_loader, boots_loader]

# data just for evaluation; in real life we do not know this mixture. We can change this composition ratio or use other loaders
# for example, this uses shirt and boots in a ratio of 30% and 70% in the evaluation task.
validaton_dataloader = sample_from([shirt_loader, boots_loader], mixing_ratio=[evaluation_task_data_ratio, 1-evaluation_task_data_ratio], base_number_of_batches=30)

iterations=10
# run BO on different mixtures of the training data. At each iteration, we train a model using the training data mixture and check the evaluation performance on validation dataset
run_BO(train_loaders, validaton_dataloader, iterations, printout=True) # should print a ratio that is close to evaluation_task_data_ratio

